In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import utils as ut
random_seed = 42

In [10]:
train_features = pd.read_csv("DOTA2_TRAIN_features.csv")
train_targets = pd.read_csv("DOTA2_TRAIN_targets.csv")
train = train_features.copy()
train["target"] = train_targets["radiant_win"]
X_train,Y_train, X_test, Y_test = ut.preprocess_train(train, True)

In [11]:
X_train

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,d5_stuns,d5_creeps_stacked,d5_camps_stacked,d5_rune_pickups,d5_firstblood_claimed,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed
7676,675.0,22.0,7.0,1.0,0.0,74.0,0.0,1.0,0.0,10.0,...,12.997,0.0,0.0,2.0,0.0,0.667,0.0,0.0,0.0,0.0
1257,203.0,22.0,7.0,1.0,7.0,19.0,0.0,0.0,0.0,4.0,...,2.433,0.0,0.0,1.0,0.0,0.167,0.0,0.0,1.0,0.0
24896,1129.0,22.0,7.0,4.0,10.0,5.0,2.0,7.0,6.0,5.0,...,11.331,0.0,0.0,4.0,0.0,0.312,1.0,0.0,0.0,0.0
7056,1768.0,22.0,7.0,8.0,7.0,120.0,1.0,4.0,6.0,11.0,...,0.000,3.0,2.0,3.0,0.0,0.400,0.0,1.0,0.0,0.0
21579,1489.0,22.0,7.0,8.0,10.0,88.0,2.0,5.0,2.0,4.0,...,10.631,15.0,4.0,8.0,0.0,0.409,0.0,0.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23444,2434.0,22.0,0.0,16.0,0.0,91.0,2.0,10.0,20.0,0.0,...,1.733,0.0,0.0,2.0,0.0,0.591,0.0,0.0,9.0,2.0
9392,1040.0,22.0,7.0,4.0,12.0,12.0,8.0,4.0,7.0,6.0,...,7.598,0.0,0.0,0.0,0.0,0.158,0.0,0.0,0.0,0.0
1499,2102.0,22.0,7.0,10.0,23.0,41.0,4.0,12.0,3.0,2.0,...,0.000,0.0,0.0,6.0,1.0,0.500,0.0,0.0,11.0,14.0
27569,1148.0,22.0,7.0,4.0,4.0,62.0,1.0,7.0,7.0,0.0,...,0.000,3.0,1.0,4.0,0.0,0.188,0.0,0.0,2.0,3.0


In [12]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
parameters = {
    'alpha' : [0.01, 0.1, 1, 2, 3, 5, 10, 50, 100],
}
#l = LinearRegression()
model = LogisticRegression()
model.fit(X_train, Y_train)

c:\Users\Oleg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:1179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Oleg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.096e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\Oleg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:1179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Oleg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix 

GridSearchCV(cv=5, estimator=RidgeClassifier(),
             param_grid={'alpha': [0.01, 0.1, 1, 2, 3, 5, 10, 50, 100]})

In [13]:
model.best_params_

{'alpha': 100}

In [14]:
from sklearn.metrics import roc_auc_score
print(f"Score : {model.score(X_test, Y_test)}")
print(f"ROC-AUC score : {roc_auc_score(Y_test, model.predict_proba(X_test))[:1]}")


Score : 0.7172066687637622


AttributeError: 'RidgeClassifier' object has no attribute 'predict_proba'

In [ ]:
ut.write(model)

[0.37799978 0.3951319  0.55269517 ... 0.50679307 0.60683847 0.12760876]
